<a href="https://colab.research.google.com/github/knagasaki/IIIF_DL/blob/master/NDLOCR_googlecolabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. GPUの情報を確認する

In [3]:
!nvidia-smi

Sat Apr 30 09:25:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)

In [4]:
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli

Cloning into 'ndlocr_cli'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 114 (delta 61), reused 33 (delta 9), pack-reused 0
Receiving objects: 100% (114/114), 66.60 KiB | 6.05 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Submodule 'src/deskew_HT' (https://github.com/ndl-lab/deskew_HT.git) registered for path 'src/deskew_HT'
Submodule 'src/ndl_layout' (https://github.com/ndl-lab/ndl_layout.git) registered for path 'src/ndl_layout'
Submodule 'src/separate_pages_ssd' (https://github.com/ndl-lab/separate_pages_ssd.git) registered for path 'src/separate_pages_ssd'
Submodule 'src/text_recognition' (https://github.com/ndl-lab/text_recognition.git) registered for path 'src/text_recognition'
Cloning into '/content/ndlocr_cli/src/deskew_HT'...
remote: Enumerating objects: 35, done.        
remote: Counting objects: 100% (35/35), done.        
remote: Compressing objects: 100% (35/35

# 2. 必要なパッケージをインストールする

In [5]:
PROJECT_DIR="/content/ndlocr_cli"

In [6]:
!pip install -r {PROJECT_DIR}/requirements.txt
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
!pip install --upgrade numpy

     |████████████████████████████████| 312 kB 11.8 MB/s 
     |████████████████████████████████| 299 kB 46.7 MB/s 
     |████████████████████████████████| 1.5 MB 46.3 MB/s 
     |████████████████████████████████| 14.8 MB 49.9 MB/s 
     |████████████████████████████████| 50.4 MB 92 kB/s 
     |████████████████████████████████| 26.5 MB 57.7 MB/s 
     |████████████████████████████████| 31.2 MB 91 kB/s 
     |████████████████████████████████| 3.1 MB 40.3 MB/s 
     |████████████████████████████████| 109.3 MB 56 kB/s 
     |████████████████████████████████| 488 kB 51.5 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Att

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:13:33tcmalloc: large alloc 1147494400 bytes == 0x2b6c000 @  0x7fe212055615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:10:56tcmalloc: large alloc 1434370048 bytes == 0x471c2000 @  0x7fe212055615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████▋          | 1336.2 MB 1.4 MB/s eta 0:07:53tcmalloc: large alloc 1792966656 byt

In [7]:
%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
!pip install dist/*.whl
%cd /content

/content/ndlocr_cli/src/ndl_layout/mmdetection
running bdist_wheel
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
running build
running build_py
creating build
creating build/lib
creating build/lib/mmdet
copying mmdet/__init__.py -> build/lib/mmdet
copying mmdet/version.py -> build/lib/mmdet
creating build/lib/mmdet/apis
copying mmdet/apis/test.py -> build/lib/mmdet/apis
copying mmdet/apis/__init__.py -> build/lib/mmdet/apis
copying mmdet/apis/train.py -> build/lib/mmdet/apis
copying mmdet/apis/inference.py -> build/lib/mmdet/apis
creating build/lib/mmdet/utils
copying mmdet/utils/logger.py -> build/lib/mmdet/utils
copying mmdet/utils/profiling.py -> build/lib/mmdet/utils
copying mmdet/utils/__init__.py -> build/lib/mmdet/utils
copying mmdet/utils/util_mixins.py

# 4. OCRに必要な学習済みモデルをダウンロードする

In [8]:
%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
%cd /content/

/content/ndlocr_cli
--2022-04-30 09:32:12--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 52.222.149.4, 52.222.149.17, 52.222.149.78, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|52.222.149.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28088 (27K) [text/plain]
Saving to: ‘./src/text_recognition/models/mojilist_NDL.txt’

mojilist_NDL.txt    100%[===================>]  27.43K  --.-KB/s    in 0s      

2022-04-30 09:32:13 (290 MB/s) - ‘./src/text_recognition/models/mojilist_NDL.txt’ saved [28088/28088]

--2022-04-30 09:32:13--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 52.222.149.4, 52.222.149.17, 52.222.149.78, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|52.222.149.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191768312 (183M) [application/x-www-form-urlencoded]
Saving to:

# 5. 環境変数を追加する

In [9]:
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

# 6. PDFを画像に変換するためのパッケージのインストール

In [10]:
!apt-get install poppler-utils
!pip install pdf2image 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (2,464 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155202 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


# 7. テキスト化したいPDFをダウンロードする
今回は、ROIS-DS人文学オープンデータ共同利用センター(http://codh.rois.ac.jp/
)が提供している

近代雑誌データセット　http://codh.rois.ac.jp/modern-magazine/
から、

東洋学芸雑誌(https://dglb01.ninjal.ac.jp/ninjaldl/bunken.php?title=toyogakuge)

第一号(https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf)

をダウンロードしてみます。

In [12]:
!mkdir -p /content/kdzk/img
import urllib.request
import json
import sys

url = 'https://www.dl.ndl.go.jp/api/iiif/1210630/manifest.json' #'http://www2.dhii.jp/nijl/NIJL0018/099-0014/manifest.json'
readObj = urllib.request.urlopen(url)
data = json.loads(readObj.read().decode('utf-8'))

arImageUrls = []
for seq in data['sequences']:
     for canvas in seq['canvases']:
         for image in canvas['images']:
             arImageUrls.append(image['resource']['service']['@id'])

tLen = len(str(len(arImageUrls))) + 1
fileNumber = 1
print ('downloading...')
for imageUrl in arImageUrls:
    localFilename = "/content/kdzk/img/"+str(fileNumber).zfill(tLen)+'.jpg'
    imageUrl = imageUrl+'/full/full/0/default.jpg'
    urllib.request.urlretrieve(imageUrl, localFilename)
    fileNumber = fileNumber + 1
    print (localFilename)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.1M  100 14.1M    0     0  4013k      0  0:00:03  0:00:03 --:--:-- 4013k
downloading...
/content/kdzk/0001.jpg
/content/kdzk/0002.jpg
/content/kdzk/0003.jpg
/content/kdzk/0004.jpg
/content/kdzk/0005.jpg
/content/kdzk/0006.jpg
/content/kdzk/0007.jpg
/content/kdzk/0008.jpg
/content/kdzk/0009.jpg
/content/kdzk/0010.jpg
/content/kdzk/0011.jpg
/content/kdzk/0012.jpg
/content/kdzk/0013.jpg
/content/kdzk/0014.jpg
/content/kdzk/0015.jpg
/content/kdzk/0016.jpg
/content/kdzk/0017.jpg
/content/kdzk/0018.jpg
/content/kdzk/0019.jpg
/content/kdzk/0020.jpg
/content/kdzk/0021.jpg
/content/kdzk/0022.jpg
/content/kdzk/0023.jpg
/content/kdzk/0024.jpg
/content/kdzk/0025.jpg
/content/kdzk/0026.jpg
/content/kdzk/0027.jpg
/content/kdzk/0028.jpg
/content/kdzk/0029.jpg
/content/kdzk/0030.jpg
/content/kdzk/0031.jpg
/content/kdzk/0032.jpg
/content/kdz

# 8. PDFをjpeg画像に変換する

# 9. OCRの実行

/content/tygz-001以下のimgディレクトリ内の画像を処理し、
/content/tygz-001_outputに出力する場合

In [16]:
%cd {PROJECT_DIR}
!python main.py infer /content/kdzk /content/kdzk_output -s s -x

ストリーミング出力は最後の 5000 行に切り捨てられました。
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '1158', 'Y': '1023', 'WIDTH': '28', 'HEIGHT': '693', 'CONF': '0.998'}
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '694', 'Y': '1027', 'WIDTH': '28', 'HEIGHT': '633', 'CONF': '0.997'}
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '1848', 'Y': '1016', 'WIDTH': '28', 'HEIGHT': '754', 'CONF': '0.996'}
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '1039', 'Y': '1020', 'WIDTH': '28', 'HEIGHT': '821', 'CONF': '0.996'}
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '1390', 'Y': '1026', 'WIDTH': '27', 'HEIGHT': '755', 'CONF': '0.995'}
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '1964', 'Y': '1010', 'WIDTH': '29', 'HEIGHT': '786', 'CONF': '0.995'}
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '1733', 'Y': '1014', 'WIDTH': '27', 'HEIGHT': '819', 'CONF': '0.995'}
This BLOCK elemetn will be skipped.
{'TYPE': 'ルビ', 'X': '924', 'Y': '1024', 'WIDTH': '2

# 10. 結果の確認

In [ ]:
import glob
import os
for fpath in sorted(glob.glob("/content/tygz-001_output/tygz-001/txt/*_main.txt")):
    with open(fpath) as f:
        txtdata=f.read()
        print(os.path.basename(fpath).replace("_main.txt",""))
        print(txtdata)